### Modelo para ML: preparacion inicial

In [ ]:
import pandas as pd

#### Paso 1: Cargar los Archivos en DataFrames

In [ ]:
# Leer los archivos Excel
ventas = pd.read_excel("../datasets/ventas.xlsx")
detalle_ventas = pd.read_excel("../datasets/detalle_ventas.xlsx")
productos = pd.read_excel("../datasets/productos.xlsx")
clientes = pd.read_excel("../datasets/clientes.xlsx")

#### Paso 2: Crear Dimensiones Normalizadas

In [ ]:
# Dimensión Productos
dim_productos = productos.drop_duplicates(subset="id_producto").copy()
dim_productos.head()

In [ ]:
# Dimensión Clientes
dim_clientes = clientes.drop_duplicates(subset="id_cliente").copy()
dim_clientes.head()

In [ ]:
# Dimension Calendario
ventas['fecha'] = pd.to_datetime(ventas['fecha'])
dim_calendario = ventas[['fecha']].drop_duplicates().copy()
dim_calendario['año'] = dim_calendario['fecha'].dt.year
dim_calendario['mes'] = dim_calendario['fecha'].dt.month
dim_calendario['día'] = dim_calendario['fecha'].dt.day
dim_calendario.head()

#### Paso 3: Crear tabla de hechos

In [ ]:
# Unimos las tablas 'ventas' y 'detalle_ventas' para crear la tabla de hechos
hechos_ventas = pd.merge(detalle_ventas, ventas, on="id_venta", how="inner")
hechos_ventas.head()

# porque usa "inner" en lugar de "left"?
# - Solo conserva las filas donde id_venta existe en ambas tablas: detalle_ventas y ventas.
# - Si hay una venta en ventas sin detalle, se excluye.
# - Si hay un detalle sin venta registrada, también se excluye.

In [ ]:
# Eliminar columnas redundantes que ya están en las dimensiones
hechos_ventas = hechos_ventas.drop(columns=["nombre_cliente", "email", "nombre_producto"])

# Convertir la columna fecha a tipo datetime
hechos_ventas["fecha"] = pd.to_datetime(hechos_ventas["fecha"])

In [ ]:
# Creación de clave primaria para la tabla de hechos
hechos_ventas = hechos_ventas.reset_index(drop=True)
hechos_ventas["id_hecho"] = hechos_ventas.index
hechos_ventas.head()

#### Paso 4: DataFrame final para ML

In [ ]:
# Crear DataFrame final para ML
df_final = hechos_ventas.merge(dim_productos, on="id_producto", how="left") \
            .merge(dim_clientes, on="id_cliente", how="left") \
            .merge(dim_calendario, left_on="fecha", right_on="fecha", how="left")

In [ ]:
df_final.columns

In [ ]:
# Conservamos el precio unitario de detalle_ventas para ML
df_ml = df_final.copy()
df_ml = df_ml.drop(columns=["precio_unitario_y"])
df_ml = df_ml.rename(columns={"precio_unitario_x": "precio_unitario"})
df_ml.head()

In [ ]:
df_ml.columns

In [ ]:
# Reordenar columnas para mejor visualización en funcion de las dimensiones
column_order = [
    "id_hecho",
    "id_venta", "fecha", "año", "mes", "día",
    "id_cliente", "nombre_cliente", "ciudad", "email", "fecha_alta",
    "id_producto", "nombre_producto", "categoria",
    "cantidad", "precio_unitario", "importe", "medio_pago"
]
df_ml = df_ml[column_order]
df_ml.head()

In [ ]:
df_ml.info()